# Data-driven prediction of battery cycle life before capacity degradation

_Zoé Marquis & Charlotte Kruzic_

## 1. Introduction 
### Présentation du contexte 

D'après la documentation : 

#### Caractéristiques des données
- Batteries : 124 cellules lithium-ion testées jusqu'à leur défaillance
- Conditions de cycle : 
    - Politiques de charge rapide au format : “C1(Q1)-C2”.
    - Tensions limites : 2,0V (inf) et 3,6V (sup)
    - Décharge à un taux de 4C
- Batches : Trois groupes distincts de données avec des spécificités propres
- Mesures disponibles : 
    - SOH
    - Résistance interne (à 30% SOC)
    - Température (susceptible de variations et de bruit)

### Objectifs

## 2. Exploration des données
### Chargement et description des données


In [ ]:
# combiner les données des 3 batchs

### Analyse exploratoire 
### Préparation des données

In [ ]:
# corriger erreurs de temps qui se réinit à zéro en milieu de cycle 

In [ ]:
# visualiser évolution 
# calcul du SOH si nécessaire
# transformation : valeurs maquantes (interpolation ou imputation), normalisation...

In [ ]:
# Ajouter des variables telles que :
# Température moyenne par cycle.
# Résistance interne.
# Taux de charge et décharge.
# Extraire les informations des politiques de charge rapide (“C1(Q1)-C2”).

In [ ]:
# split train test

## 3. Modélisation 
### Modèle 1 : 
### Modèle 2 : 
### Modèle 3 : 

In [ ]:
# Modèles basiques : Régression linéaire, Random Forest.
# Modèles avancés : Réseaux de neurones récurrents (RNN, LSTM, GRU).

# Régression linéaire : Point de départ pour comprendre la relation linéaire.
# Random Forest : Modèle robuste pour capturer les non-linéarités.
# RNN / LSTM : Adapter pour les données séquentielles, avec gestion des fenêtres temporelles.

# Comparer les résultats avec des métriques :
# RMSE : Erreur quadratique moyenne.
# MAE : Erreur absolue moyenne.
# R² : Coefficient de détermination.

## 4. Comparaison des métriques pour évaluer la performance des différents modèles

In [ ]:
# Optimisation : 
# Ajuster les hyperparamètres des modèles pour améliorer les performances (Grid Search, Random Search).
# Analyser les temps de calcul et optimiser le code si nécessaire.

## 5. Explicabilité
### Application des outils d'explicabilité
### Discussion des résultats

## 6. Conclusion
### Résumé des performances des modèles
### Interprétation des résultats d'explicabilité
### Perspectives : comment utiliser les résultats pour guider la maintenance et l’optimisation des batteries